# Classification de texte avec un réseau de neurones Keras

Ce notebook détaille un pipeline complet de classification de texte avec Keras/TensorFlow, du prétraitement à la prédiction. Chaque étape est expliquée, y compris les principes mathématiques et les choix algorithmiques.

---

## Concepts clés

- **Prétraitement linguistique** : Nettoyage, lemmatisation, suppression des stopwords pour améliorer la qualité des features.
- **Tokenization & Padding** : Transformation du texte en séquences d'indices, puis normalisation de la longueur des séquences.
- **Embedding** : Apprentissage de représentations vectorielles denses pour chaque mot.
- **GlobalAveragePooling1D** : Réduction d'une séquence de vecteurs en un seul vecteur par moyenne.
- **Dense & Dropout** : Couches du réseau de neurones pour la classification et la régularisation.
- **Softmax** : Fonction d'activation pour la classification multi-classes.
- **Categorical Crossentropy** : Fonction de perte adaptée à la classification multi-classes.

---

In [ ]:
# Importation des bibliothèques nécessaires
import pandas as pd
import re
import spacy
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Embedding, GlobalAveragePooling1D
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

## Chargement et prétraitement du texte

On utilise SpaCy pour nettoyer et lemmatiser les textes :
- Suppression du HTML et des URLs
- Mise en minuscules
- Lemmatisation (réduction à la racine)
- Suppression des stopwords et tokens non alphabétiques

Ce prétraitement réduit la dimensionnalité et améliore la qualité des features pour le modèle.

In [ ]:
print("Loading SpaCy model...")
nlp = spacy.load('en_core_web_sm')

def clean_and_tokenize(text):
    if not isinstance(text, str) or not text.strip():
        return ""
    text = re.sub(r'<[^>]+>', ' ', text)
    text = re.sub(r'http\S+', ' ', text)
    text = text.lower().strip()
    doc = nlp(text)
    tokens = [token.lemma_ for token in doc if token.is_alpha and not token.is_stop]
    return " ".join(tokens)

## Chargement du jeu de données et nettoyage

On charge les données d'entraînement, puis on applique le nettoyage et la lemmatisation à chaque description.

In [ ]:
version_ = "V.3.0.6"
print("Version:", version_)
print("Loading dataset...")
data = pd.read_json('train_mini.json').set_index('Id')
print("Dataset loaded successfully!")

print("Preprocessing data...")
data['Cleaned_Description'] = data['description'].apply(clean_and_tokenize)
X = data['Cleaned_Description']
print(f"Number of samples: {len(X)}")

## Mapping des descriptions vers les catégories

On utilise un fichier de mapping pour associer chaque description nettoyée à une catégorie métier. Cela permet de transformer le problème en classification supervisée.

In [ ]:
print("Loading job categories...")
categories_mapping = pd.read_csv('categories_string.csv', header=None, index_col=0, squeeze="columns").to_dict()

print("Mapping descriptions to job categories...")
data['Category'] = data['Cleaned_Description'].map(categories_mapping)

unmapped = data[data['Category'].isnull()]
if not unmapped.empty:
    print("Warning: Some descriptions could not be mapped. Unmapped values:")
    print(unmapped[['description', 'Cleaned_Description']])

data = data.dropna(subset=['Category'])
if data.empty:
    raise ValueError("No valid data left after mapping categories. Check your input files.")
print("Categories mapped successfully!")

## Encodage des catégories (labels)

On transforme les catégories textuelles en entiers avec `LabelEncoder`, puis en one-hot avec `to_categorical` pour la classification multi-classes.

Mathématiquement, chaque label $y$ est transformé en un vecteur de taille $C$ (nombre de classes), où la case correspondant à la classe est à 1, les autres à 0.

In [ ]:
print("Encoding job categories...")
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(data['Category'])
y_categorical = to_categorical(y_encoded)
print("Job categories encoded successfully!")

## Tokenization et padding des textes

On transforme chaque texte en une séquence d'indices de mots (`Tokenizer`), puis on tronque ou complète chaque séquence à une longueur fixe (`pad_sequences`).

Cela permet d'obtenir une matrice $(n_{samples}, maxlen)$ où chaque ligne représente un texte sous forme d'indices.

In [ ]:
print("Tokenizing and padding text data...")
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(X)
X_tokenized = tokenizer.texts_to_sequences(X)
X_padded = pad_sequences(X_tokenized, maxlen=100)
print("Text data tokenized and padded successfully!")

## Construction du modèle de réseau de neurones

Le modèle comprend :
- Une couche d'embedding (chaque mot est représenté par un vecteur dense appris pendant l'entraînement)
- Un GlobalAveragePooling1D (fait la moyenne des vecteurs de mots pour obtenir une représentation globale du texte)
- Une couche dense cachée avec activation ReLU
- Un dropout (régularisation)
- Une couche de sortie softmax (probabilité pour chaque classe)

Mathématiquement, le modèle apprend une fonction $f(x) = \text{softmax}(W_2 \cdot \text{ReLU}(W_1 \cdot \text{mean}(E(x)) + b_1) + b_2)$ où $E(x)$ est la séquence des embeddings du texte.

In [ ]:
print("Building the neural network model...")
model = Sequential([
    Embedding(input_dim=5000, output_dim=128, input_length=100),
    GlobalAveragePooling1D(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(y_categorical.shape[1], activation='softmax')
])
print("Model built successfully!")

## Compilation du modèle

- **Adam** : optimiseur adaptatif efficace pour l'entraînement des réseaux de neurones.
- **Categorical Crossentropy** : fonction de perte adaptée à la classification multi-classes.
- **Accuracy** : métrique d'évaluation (proportion de bonnes prédictions).

In [ ]:
print("Compiling the model...")
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
print("Model compiled successfully!")

## Entraînement du modèle

On entraîne le modèle sur toutes les données disponibles. À chaque époque, le modèle ajuste ses poids pour minimiser la perte.

La descente de gradient ajuste les poids pour maximiser la probabilité des bonnes classes.

In [ ]:
print("Training the model on all available data...")
model.fit(X_padded, y_categorical, epochs=10, batch_size=32, verbose=1)
print("Model training completed!")

## Préparation et prétraitement des données de test

On applique le même nettoyage, tokenization et padding aux textes de test.

In [ ]:
print("Loading test dataset...")
test_data = pd.read_json('test_mini.json').set_index('Id')
print(f"Number of test samples: {len(test_data)}")

print("Preprocessing test data...")
test_data['Cleaned_Description'] = test_data['description'].apply(clean_and_tokenize)
X_test = test_data['Cleaned_Description']
X_test_tokenized = tokenizer.texts_to_sequences(X_test)
X_test_padded = pad_sequences(X_test_tokenized, maxlen=100)
print("Test data preprocessed successfully!")

## Prédiction sur les données de test

Le modèle prédit la probabilité d'appartenance à chaque classe pour chaque texte. On prend la classe avec la probabilité maximale.

In [ ]:
print("Generating predictions for test data...")
predictions = model.predict(X_test_padded)
predicted_classes = predictions.argmax(axis=1)
print("Predictions generated successfully!")

## Décodage des prédictions

On utilise le label encoder pour retrouver les noms de catégories d'origine à partir des indices prédits.

In [ ]:
print("Mapping predictions to job categories...")
predicted_categories = label_encoder.inverse_transform(predicted_classes)
print("Predictions mapped to job categories successfully!")

## Génération du fichier de soumission

On prépare le fichier de soumission au format attendu, associant chaque Id de test à la catégorie prédite.

In [ ]:
print("Saving predictions to submissions3.csv...")
submissions3 = pd.DataFrame({
    'Id': test_data.index,
    'Category': predicted_categories
})
submissions3.to_csv('submissions3.csv', index=False)
print("Predictions saved to submissions3.csv successfully!")

---

## Concepts mathématiques et conclusion

- **Embedding** : chaque mot est représenté par un vecteur dense appris pendant l'entraînement. L'embedding permet de capturer la similarité sémantique entre mots.
- **GlobalAveragePooling1D** : fait la moyenne des embeddings pour obtenir une représentation globale du texte.
- **Dense & Softmax** : la couche dense calcule une combinaison linéaire des features, softmax transforme ces scores en probabilités.
- **Categorical Crossentropy** : pour chaque exemple, la perte est $-\sum_{c} y_c \log(p_c)$ où $y_c$ est la vraie classe (one-hot) et $p_c$ la probabilité prédite.

Ce pipeline montre comment passer de textes bruts à des prédictions de classes avec un réseau de neurones, en utilisant des représentations vectorielles apprises automatiquement.